In [1]:
from enum import Enum
from abc import abstractmethod
 
#######################################################################################
#   Instructioncodes that can be used to determine also Direction, and turning calcs. #
#######################################################################################


class InstructionCode(Enum):
    pass

class TurnInstructionCode(InstructionCode):
    L = 0
    R = 1
    
class MoveInstructionCode(Enum):
    '''This instruction code is only here for completeness, it does not serve an actual purpose.
    I wanted to make sure in part 1 that, there was no need for this later on.'''
    F = 0
    
class DirectionInstructionCode(Enum):
    '''Turning the ship only requires a modulo operation this way, since python allows negative modulo'''
    N = 0
    E = 1
    S = 2
    W = 3

    
    
#######################################################################################
# The instructions that apply transformations on the systemstate via the apply method.#
#######################################################################################


class Instruction():
    
    def __init__(self, instructioncode, value):
        self.instructioncode = instructioncode
        self.value = value
        
    @abstractmethod
    def apply1(self, position, direction):
        '''Rules for part 1'''
        pass
    
    @abstractmethod
    def apply2(self, shippos, shipdir, waypos):
        '''Rules for part 2'''
        pass
    
    
    
class TurnInstruction(Instruction):
    
    def __init__(self, instructioncode, value):
        '''Mapping the values given in degrees to the code of Directions (relative)'''
        super(TurnInstruction, self).__init__(instructioncode, value)
        self.value = int(value / 90)
    
    def apply1(self, position, direction): 
        '''Since the values within turn instruction are also DirectionInstructionCodes modulo can be abused.'''
        if self.instructioncode is TurnInstructionCode.L:
            directionvalue = (direction.value - self.value) % 4
        elif self.instructioncode is TurnInstructionCode.R:
            directionvalue = (direction.value + self.value) % 4
        return position, DirectionInstructionCode(directionvalue)
    
    def apply2(self, shippos, shipdir, waypos):
        '''Simply transforming the coordinate acoording to the turning. This could be optimized.'''
        # This part looks very ugly but i got tired of thinking about turning the coordinate system.
        if self.instructioncode is TurnInstructionCode.L:
            if self.value == 0:
                newwaypos = waypos
            elif self.value == 1:
                newwaypos = (-waypos[1], waypos[0])
            elif self.value == 2:
                newwaypos = (-waypos[0], -waypos[1])
            elif self.value == 3:
                newwaypos = (waypos[1], -waypos[0])
        elif self.instructioncode is TurnInstructionCode.R:  
            if self.value == 0:
                newwaypos = waypos
            elif self.value == 1:
                newwaypos = (waypos[1], -waypos[0])
            elif self.value == 2:
                newwaypos = (-waypos[0], -waypos[1])
            elif self.value == 3:
                newwaypos = (-waypos[1], waypos[0])
        return shippos, shipdir, newwaypos
        
        
            
class DirectionInstruction(Instruction):
        
    def apply1(self, position, direction):
        '''Moving a coordinate according to the instruction.'''
        if self.instructioncode is DirectionInstructionCode.N:
            newposition = (position[0], position[1] + self.value)
        elif self.instructioncode is DirectionInstructionCode.E:
            newposition = (position[0] + self.value, position[1])
        elif self.instructioncode is DirectionInstructionCode.S:
            newposition = (position[0], position[1] - self.value)
        elif self.instructioncode is DirectionInstructionCode.W:
            newposition = (position[0] - self.value, position[1])
        return newposition, direction
    
    def apply2(self, shippos, shipdir, waypos):
        '''This uses the method form part 1 by simply applying it to the waypoint instead.'''
        return shippos, shipdir, self.apply1(waypos, None)[0]
    
    
    
    
class MoveInstruction(Instruction):
    
    def __init__(self, value):
        '''Nicer looking constructor. The MoveInstructionCode is actually quite useless'''
        super(MoveInstruction, self).__init__(MoveInstructionCode.F, value)
    
    def apply1(self, position, direction):
        '''A move instruction in part 1 is simply a direction instruction, in the ships direction.
        Also using the fact, that the DirectionInstructionCode and the ships direction are encoded
        by the same datatype here.'''
        instruction = DirectionInstruction(direction, self.value)
        return instruction.apply1(position, direction)
    
    def apply2(self, shippos, shipdir, waypos):
        '''A move instruction in part 2 is a combination of two DirectionInstructions from part 1.
        This way I can simply extract the direction from the paypoint and build two 
        DirectionInstructions accordingly.'''
        newshippos = shippos
        for _ in range(self.value):
            if waypos[0] != 0:
                if waypos[0] > 0:
                    horizontalinstruction = DirectionInstruction(DirectionInstructionCode.E, waypos[0])
                elif waypos[0] < 0:
                    horizontalinstruction = DirectionInstruction(DirectionInstructionCode.W, -waypos[0])
                newshippos, shipdir = horizontalinstruction.apply1(newshippos, shipdir)
            if waypos[1] != 0:
                if waypos[1] > 0:
                    verticalinstruction = DirectionInstruction(DirectionInstructionCode.N, waypos[1])
                elif waypos[1] < 0:
                    verticalinstruction = DirectionInstruction(DirectionInstructionCode.S, -waypos[1])
                newshippos, shipdir = verticalinstruction.apply1(newshippos, shipdir)
        return newshippos, shipdir, waypos
       
        
        
#######################################################################################
#   Utility classes that are there for completeness of OOD. Also made thinking easier.#
#######################################################################################


class InstructionBuilder():
    '''This class exists for easy transformation of the Strings to actual instructions.
    Using the fact, that enumeration names are the named the same as the string values here.'''
    
    def build(self, instructioncodechar, value):
        if instructioncodechar in ['N', 'S', 'E', 'W']:
            instructioncode = DirectionInstructionCode[instructioncodechar]
            return DirectionInstruction(instructioncode, value)
        elif instructioncodechar in ['L', 'R']:
            instructioncode = TurnInstructionCode[instructioncodechar]
            return TurnInstruction(instructioncode, value)
        elif instructioncodechar is 'F':
            instructioncode = MoveInstructionCode[instructioncodechar]
            return MoveInstruction(value)
        
        
        
class Ship():
    '''A simple class that executes the instructions it's given in order'''
    
    def __init__(self):
        self.position = (0,0)
        self.direction = DirectionInstructionCode.E
        self.waypos = (10, 1)
        
    def _apply1(self, instruction):
        self.position, self.direction = instruction.apply1(self.position, self.direction)
        
    def _apply2(self, instruction):
        self.position, self.direction, self.waypos = instruction.apply2(self.position, self.direction, self.waypos)
        
    def run1(self, instructionset):
        for instruction in instructionset:
            self._apply1(instruction)
        return self.position, self.direction
    
    def run2(self, instructionset):
        for instruction in instructionset:
            self._apply2(instruction)
        return self.position, self.direction, self.waypos         
    
    
    
def distance(pos):
    from scipy.spatial.distance import cityblock
    return cityblock(pos, (0,0))



filename = "./input.txt"
with open(filename) as f:
    content = [line.strip() for line in f.readlines()]

instructionbuilder = InstructionBuilder()
instructionset = [instructionbuilder.build(instructionstring[0], int(instructionstring[1:])) for instructionstring in content]
ship = Ship()
endposition, enddirection = ship.run1(instructionset)
print("Solution part 1: " + str(distance(endposition)))

ship = Ship()
endposition, enddirection, waypos = ship.run2(instructionset)
print("Solution part 2: " + str(distance(endposition)))


Solution part 1: 2458
Solution part 2: 145117
